# Cross-Project BigQuery Access: IAM Permissions Lab

This notebook demonstrates how to configure and troubleshoot IAM permissions when a Vertex AI Pipeline needs to access BigQuery tables across different GCP projects.

## Architecture Overview

**Project 1** (`matt-demos`): Pipeline execution project where Vertex AI Pipeline runs  
**Project 2** (`matt-demos-secondary`): Data storage project containing the BigQuery table  
**Service Account**: `bby-lab@matt-demos.iam.gserviceaccount.com`

## Required IAM Roles

### Project 1: Pipeline Project (`matt-demos`)
Service Account: `bby-lab@matt-demos.iam.gserviceaccount.com`
- BigQuery Job User (create and run query jobs)
- Storage Object Admin (access pipeline artifacts)
- Vertex AI User (submit and manage pipelines)

### Project 2: Data Project (`matt-demos-secondary`)
Service Account: `bby-lab@matt-demos.iam.gserviceaccount.com`
- BigQuery Data Viewer (read table data)

## Cross-Project BigQuery Access Flow

1. **Pipeline Start**: Vertex AI Pipeline initiates in Project 1
2. **Component Initialization**: Pipeline component begins execution
3. **Client Setup**: BigQuery client initialized with billing project set to Project 1
4. **Job Submission**: Query job submitted to Project 1 (requires `bigquery.jobs.create`)
5. **SQL Execution**: Query references table in Project 2
   ```sql
   SELECT * FROM `matt-demos-secondary.dataset_id.table_id`
   ```
6. **Permission Check**: BigQuery validates service account permissions in Project 2
7. **Data Transfer**: Data streams from Project 2 to Project 1 pipeline component
8. **Completion**: Component processes data and completes

## Key Concepts

- The **billing project** (Project 1) is where query jobs are executed and billed
- The **data project** (Project 2) is where the source table resides
- The service account requires appropriate permissions in **both projects**

In [ ]:
# Install required dependencies for Kubeflow Pipelines, BigQuery, and Vertex AI
%pip install --upgrade kfp google-cloud-bigquery \
    bigframes google-cloud-aiplatform \
    db-dtypes google-cloud-pipeline-components

In [1]:
# Import required libraries
import os
import time
from typing import NamedTuple
from datetime import datetime, timedelta
import random
import logging

# Kubeflow Pipelines SDK
from kfp import dsl, compiler

# Google Cloud services
from google.cloud import aiplatform
from google.cloud import bigquery
import bigframes.pandas as bpd

In [2]:
# Configuration parameters for the pipeline and BigQuery resources

# Project configuration
PROJECT_ID = "matt-demos"  # Primary project where pipeline executes
PROJECT_ID_2 = "matt-demos-secondary"  # Secondary project containing BigQuery data
LOCATION = "us-central1"  # GCP region for pipeline execution

# Storage configuration
STAGING_BUCKET = "gs://b1bd1e40-2c20-433d-83a1-9b691be93b38"  # GCS bucket for pipeline artifacts
PIPELINE_ROOT = os.path.join(STAGING_BUCKET, "pipeline-root")

# BigQuery table configuration
DATASET_ID = "synthetic_data"  # Dataset name in Project 2
TABLE_ID = "sample_products"  # Table name in Project 2

## Setup: Create Sample BigQuery Dataset

This section creates a sample BigQuery table with synthetic product data in the secondary project for demonstration purposes.

In [ ]:
# Initialize BigQuery client for the secondary project
client = bigquery.Client(project=PROJECT_ID_2, location="US")

# Construct fully qualified table identifier
FULL_TABLE_ID = f"{PROJECT_ID_2}.{DATASET_ID}.{TABLE_ID}"

# Create dataset if it doesn't already exist
dataset = bigquery.Dataset(f"{PROJECT_ID_2}.{DATASET_ID}")
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Dataset {DATASET_ID} created or already exists in US location")

# Define schema for the products table
schema = [
    bigquery.SchemaField("product_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("product_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("category", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("in_stock", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("created_date", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
]

# Create table with defined schema
table = bigquery.Table(FULL_TABLE_ID, schema=schema)
table = client.create_table(table, exists_ok=True)
print(f"Table {FULL_TABLE_ID} created")

# Generate synthetic product data
categories = ["Electronics", "Clothing", "Books", "Home & Garden", "Toys", "Sports"]
products = [
    "Laptop", "T-Shirt", "Novel", "Chair", "Action Figure", "Basketball",
    "Smartphone", "Jeans", "Cookbook", "Lamp", "Puzzle", "Tennis Racket",
    "Tablet", "Dress", "Magazine", "Rug", "Board Game", "Golf Clubs",
    "Headphones", "Sweater", "Comic Book", "Vase", "LEGO Set", "Yoga Mat"
]

rows_to_insert = []
base_date = datetime.now() - timedelta(days=365)

# Create 100 rows of synthetic product data
for i in range(1, 101):
    product_name = random.choice(products) + f" Model-{i}"
    category = random.choice(categories)
    price = round(random.uniform(9.99, 999.99), 2)
    in_stock = random.choice([True, False])
    created_date = base_date + timedelta(days=random.randint(0, 365))
    description = f"High-quality {product_name.lower()} from the {category} category"
    
    rows_to_insert.append({
        "product_id": i,
        "product_name": product_name,
        "category": category,
        "price": price,
        "in_stock": in_stock,
        "created_date": created_date.isoformat(),
        "description": description
    })

# Insert rows into BigQuery table
errors = client.insert_rows_json(FULL_TABLE_ID, rows_to_insert)

if errors:
    print(f"Errors occurred while inserting rows: {errors}")
else:
    print(f"Successfully inserted {len(rows_to_insert)} rows into {FULL_TABLE_ID}")
    
# Query to verify data insertion and display summary statistics
query = f"""
    SELECT 
        category,
        COUNT(*) as product_count,
        AVG(price) as avg_price,
        SUM(CASE WHEN in_stock THEN 1 ELSE 0 END) as in_stock_count
    FROM `{FULL_TABLE_ID}`
    GROUP BY category
    ORDER BY product_count DESC
"""

query_job = client.query(query)
results = query_job.result()

print("\nData Summary:")
for row in results:
    print(f"Category: {row.category}, Count: {row.product_count}, Avg Price: ${row.avg_price:.2f}, In Stock: {row.in_stock_count}")

## Pipeline Definition

This section defines the Vertex AI Pipeline components and the pipeline itself. Two approaches are demonstrated for reading BigQuery data.

### Component 1: BigFrames Approach (Recommended)

Uses BigFrames for lazy evaluation and server-side processing. This approach is more efficient for large datasets as it minimizes data transfer and leverages BigQuery's distributed compute.

In [3]:
@dsl.component(base_image="python:3.12", packages_to_install=["bigframes", "db-dtypes"])
def read_bigframes_bigquery_table(
    project_id: str, 
    billing_project_id: str, 
    dataset_id: str, 
    table_id: str
) -> str:
    """
    Reads a BigQuery table using BigFrames with lazy evaluation.
    
    Args:
        project_id: GCP project containing the BigQuery table (data project)
        billing_project_id: GCP project to bill query costs to (pipeline project)
        dataset_id: BigQuery dataset name
        table_id: BigQuery table name
    
    Returns:
        String representation of the DataFrame
    """
    import logging
    import sys
    import bigframes.pandas as bpd

    # Construct fully qualified table identifier
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Configure logging to capture detailed execution information
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)

    # Helper function to ensure logs are flushed immediately
    def flush_logs():
        sys.stdout.flush()
        sys.stderr.flush()
        for handler in logger.handlers:
            handler.flush()
    
    try:
        # Configure BigFrames to use the billing project for query execution
        bpd.options.bigquery.project = billing_project_id
        bpd.options.bigquery.location = "US"

        # Construct SQL query to read entire table
        sql = f"""
            SELECT *
            FROM `{full_table_id}`
        """

        # Execute query using BigFrames (lazy evaluation)
        # Query is not executed until data is materialized
        all_features_bf = bpd.read_gbq(sql, use_cache=True)

        logger.debug(f"Shape of data loaded: {all_features_bf.shape}")
        flush_logs()

        # Convert BigFrames DataFrame to pandas (triggers actual query execution)
        all_features = all_features_bf.to_pandas()

        # Log summary information
        logger.info(f"\nLoaded {all_features.shape[0]} rows and {all_features.shape[1]} columns from {full_table_id}")
        
        # Return DataFrame as string representation
        dataframe_str = all_features.to_string()
        return dataframe_str
            
    except Exception as e:
        logger.error(f"Error reading table: {str(e)}")
        raise

In [4]:
@dsl.component(base_image="python:3.12", packages_to_install=["google-cloud-bigquery", "db-dtypes"])
def read_bigquery_table(
    project_id: str, 
    billing_project_id: str, 
    dataset_id: str, 
    table_id: str
) -> str:
    """
    Reads a BigQuery table using the traditional google-cloud-bigquery library.
    
    Args:
        project_id: GCP project containing the BigQuery table (data project)
        billing_project_id: GCP project to bill query costs to (pipeline project)
        dataset_id: BigQuery dataset name
        table_id: BigQuery table name
    
    Returns:
        String representation of the DataFrame
    """
    import logging
    import sys
    from google.cloud import bigquery

    # Construct fully qualified table identifier
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Configure logging to capture detailed execution information
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)

    # Helper function to ensure logs are flushed immediately
    def flush_logs():
        sys.stdout.flush()
        sys.stderr.flush()
        for handler in logger.handlers:
            handler.flush()
    
    try:
        # Configure query job to enable result caching
        job_config = bigquery.QueryJobConfig(use_query_cache=True)

        # Construct SQL query to read entire table
        sql = f"""
            SELECT *
            FROM `{full_table_id}`
        """

        # Initialize BigQuery client with billing project and execute query
        # Results are immediately downloaded and converted to pandas DataFrame
        all_features = bigquery.Client(
            location="US", 
            project=billing_project_id
        ).query(sql, job_config=job_config).to_dataframe()

        logger.debug(f"Shape of data loaded: {all_features.shape}")
        flush_logs()

        # Log summary information
        logger.info(f"\nLoaded {all_features.shape[0]} rows and {all_features.shape[1]} columns from {full_table_id}")
        
        # Return DataFrame as string representation
        dataframe_str = all_features.to_string()
        return dataframe_str
            
    except Exception as e:
        logger.error(f"Error reading table: {str(e)}")
        raise

In [5]:
@dsl.pipeline(
    name="bigquery-cross-project-access-pipeline", 
    pipeline_root=PIPELINE_ROOT
)
def pipeline(
    project_id: str, 
    billing_project_id: str, 
    location: str, 
    dataset_id: str, 
    table_id: str
):
    """
    Pipeline that demonstrates reading BigQuery data across projects.
    
    Args:
        project_id: Project containing the BigQuery table
        billing_project_id: Project to bill query execution to
        location: GCP region for pipeline execution
        dataset_id: BigQuery dataset name
        table_id: BigQuery table name
    """
    
    # BigFrames approach (recommended for large datasets)
    read_bigframes = read_bigframes_bigquery_table(
        project_id=project_id,
        billing_project_id=billing_project_id,
        dataset_id=dataset_id,
        table_id=table_id
    )

    # Traditional approach (simpler but less efficient for large data)
    read_traditional = read_bigquery_table(
        project_id=project_id,
        billing_project_id=billing_project_id,
        dataset_id=dataset_id,
        table_id=table_id
    )
    
# Compile the pipeline to JSON specification
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="bigquery_cross_project_access_pipeline.json",
)

In [7]:
# Initialize Vertex AI SDK with primary project configuration
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Define path to compiled pipeline specification
package_path = "bigquery_cross_project_access_pipeline.json"

# Create and configure the pipeline job
job = aiplatform.PipelineJob(
    display_name=f"bigquery-cross-project-access-{int(time.time())}",
    template_path=package_path,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "project_id": PROJECT_ID_2,  # Data project containing BigQuery table
        "billing_project_id": PROJECT_ID,  # Billing project for query execution
        "location": LOCATION,
        "dataset_id": DATASET_ID,
        "table_id": TABLE_ID
    },
    enable_caching=False,  # Disable caching to ensure fresh execution
)

# Submit the pipeline job with specified service account
# Note: Service account must have required permissions in both projects
job.submit(service_account="bby-lab@matt-demos.iam.gserviceaccount.com")

# Display job information and console link
print(f"Job name: {job.resource_name}")
print(f"Console URL: https://console.cloud.google.com/vertex-ai/pipelines/runs/{job.name.split('/')[-1]}?project={PROJECT_ID}")

Creating PipelineJob
PipelineJob created. Resource name: projects/941046250687/locations/us-central1/pipelineJobs/bigquery-cross-project-access-pipeline-20251030173541
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/941046250687/locations/us-central1/pipelineJobs/bigquery-cross-project-access-pipeline-20251030173541')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bigquery-cross-project-access-pipeline-20251030173541?project=941046250687
Job name: projects/941046250687/locations/us-central1/pipelineJobs/bigquery-cross-project-access-pipeline-20251030173541
Console URL: https://console.cloud.google.com/vertex-ai/pipelines/runs/bigquery-cross-project-access-pipeline-20251030173541?project=matt-demos


---

## Technical Reference: BigQuery Data Loading Approaches

This section provides a detailed technical comparison of the two methods for loading BigQuery data.

### Comparison Table

| Aspect | `google-cloud-bigquery` | `bigframes` |
|--------|------------------------|-------------|
| **Execution Model** | Eager | Lazy |
| **Processing Location** | Client (local Python) | Server (BigQuery) |
| **Memory Constraint** | Local RAM | None |
| **Data Transfer** | Full dataset | Requested results only |
| **Query Optimization** | Manual | Automatic (query fusion) |
| **Scalability** | RAM-limited | Cloud-scale |
| **Temporary Tables** | Created for large results | Minimized |

### Query Optimization Example

**Traditional Approach** (3 network roundtrips):
```python
df = client.query("SELECT * FROM table").to_dataframe()  # Downloads 100 rows
filtered = df[df['price'] > 100]                         # Filters in Python
result = filtered.groupby('category')['price'].mean()    # Aggregates in Python
```

**BigFrames Approach** (1 optimized query):
```python
df = bpd.read_gbq("SELECT * FROM table")              # No execution
filtered = df[df['price'] > 100]                      # Adds to computation graph
result = filtered.groupby('category')['price'].mean() # Compiles to SQL
